In [17]:
pip install -qU yfinance langchain langchain-openai

In [18]:
pip install -qU langchain-community

In [16]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

from langchain.agents import tool
import yfinance as yf

@tool
def get_stock_price(symbol: str) -> str:
    """دریافت قیمت لحظه‌ای سهام با استفاده از نماد (مثل AAPL یا GOOGL)"""
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d")
        if not data.empty:
            last_price = data['Close'].iloc[-1]
            return f"قیمت لحظه‌ای {symbol}: {last_price:.2f} دلار"
        return f"نمودار قیمتی برای نماد {symbol} یافت نشد"
    except Exception as e:
        return f"خطا در دریافت اطلاعات سهام: {str(e)}"

import os
#os.environ["OPENAI_API_KEY"] = "کلید_API_تو"
from getpass import getpass
api_key = getpass("کلید API خود را وارد کنید (متن پنهان می‌ماند): ")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

tools = [get_stock_price]

from langchain import hub
prompt = hub.pull("hwchase17/react")

from langchain.agents import create_react_agent, AgentExecutor
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=20,
    handle_parsing_errors=True
)

query_list = [
    "قیمت سهام اپل چقدر است؟",
    "آیا می‌توانی قیمت سهام مایکروسافت را به من بگویی؟",
    "وضعیت سهام گوگل چگونه است؟",
    "TSLA قیمت چقدر است؟"
]

import asyncio
async def main():
    for i, query in enumerate(query_list, start=1):
        result = agent_executor.invoke({"input": query})
        print(f"نتیجه پرسش {i}: {result}")

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


کلید API خود را وارد کنید (متن پنهان می‌ماند): ··········


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
I should use the get_stock_price tool to find the current price of Apple stock.
Action: get_stock_price
Action Input: "AAPL"قیمت لحظه‌ای AAPL: 229.13 دلارI need to provide the answer in the requested format.
Final Answer: قیمت سهام اپل در حال حاضر 229.13 دلار است.

> Finished chain.
نتیجه پرسش 1: {'input': 'قیمت سهام اپل چقدر است؟', 'output': 'قیمت سهام اپل در حال حاضر 229.13 دلار است.'}


> Entering new AgentExecutor chain...
I should use the get_stock_price tool to get the stock price of Microsoft.
Action: get_stock_price
Action Input: "MSFT"قیمت لحظه‌ای MSFT: 522.46 دلارI now have the stock price of Microsoft.
Final Answer: قیمت لحظه‌ای سهام مایکروسافت 522.46 دلار است.

> Finished chain.
نتیجه پرسش 2: {'input': 'آیا می\u200cتوانی قیمت سهام مایکروسافت را به من بگویی؟', 'output': 'قیمت لحظه\u200cای سهام مایکروسافت 522.46 دلار است.'}


> Entering new AgentExecutor chain...
I should use the get_stock_price tool to check the current price of Google